In [ ]:
%load_ext autoreload
%autoreload 2

In [ ]:
from pathlib import Path
from pprint import pprint

from huggingface_hub import snapshot_download
from hydra import compose, initialize
from hydra.core.global_hydra import GlobalHydra
from IPython.display import Video
from omegaconf import OmegaConf
from omegaconf.dictconfig import DictConfig

from examples.notebook_utils import config_notebook
from examples.pretrained_script import download_eval_pretrained
from lerobot.scripts.eval import eval

# Select policy and env
POLICY = "diffusion"  # "tdmpc" | "diffusion"
ENV = "pusht"  # "pusht" | "simxarm"

# Select device
DEVICE = "mps"  # "cuda" | "mps"

# Generated videos will be written here
OUT_DIR = Path("./outputs")
OUT_EXAMPLE = OUT_DIR / "eval" / "eval_episode_0.mp4"

PRETRAINED_REPO = "lerobot/diffusion_policy_pusht_image"
pretrained_folder = Path(snapshot_download(repo_id=PRETRAINED_REPO, repo_type="model", revision="v1.0"))
pretrained_model_path = pretrained_folder / "model.pt"

cfg_path = pretrained_folder / "config.yaml"
GlobalHydra.instance().clear()

print(pretrained_folder)

initialize(config_path="../../../.cache/huggingface/hub/models--lerobot--diffusion_policy_pusht_image/snapshots/163d168f5c193c356b82e3bf6bbf5b4eeaa780d7")
overrides = [
    f"env={ENV}",
    f"policy={POLICY}",
    f"device={DEVICE}",
    f"+policy.pretrained_model_path={pretrained_model_path}",
    f"eval_episodes=1",
    f"+env.episode_length=200",
]
cfg = compose(config_name="config", overrides=overrides)
pprint(OmegaConf.to_container(cfg))
# Setup config
#cfg = config_notebook(cfg_path, policy=POLICY, env=ENV, device=DEVICE, print_config=False, pretrained_model_path=pretrained_model_path)

In [ ]:
# eval(cfg, out_dir=OUT_DIR)
download_eval_pretrained(OUT_DIR, cfg)
Video(OUT_EXAMPLE, embed=True)